In [1]:
import pandas as pd
import numpy as np

In [32]:
items = pd.read_csv('../data/items.tsv', delimiter='\t', encoding='utf-8')
items.item_code = items.item_code.apply(lambda x: str(x).strip())
series = pd.read_csv('../data/series.tsv', delimiter='\t', encoding='utf-8')
series.rename(columns={'series_id        ': 'series_id'}, inplace=True)
series.series_id = series.series_id.apply(lambda x: str(x).strip())
fulldata = pd.read_csv('../data/fulldata.tsv', delimiter='\t', encoding='utf-8')
fulldata.rename(columns={'series_id        ': 'series_id', '       value':'value'}, inplace=True)
fulldata.series_id = fulldata.series_id.apply(lambda x: str(x).strip())

/tmp/ipykernel_171115/60638053.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  fulldata = pd.read_csv('../data/fulldata.tsv', delimiter='\t', encoding='utf-8')


In [33]:
menu = {
    'id':[], 
    'menu_name':[], 
    'item_codes':[], 
    'price_multipliers':[], 
    'category':[],
    'methodology':[]
    }

In [34]:
0.0625*2

0.125

In [35]:
drink_ids = [0, 1, 2, 3]
drink_names = ['Cup of Orange Juice', 'Cup of Milk', 'Cup of Coffee', 'Cup of Water']
drink_item_codes = [['713111'], ['709112'], ['717311'], ['FN1102']]
drink_price_multipliers = [[0.1667], [0.1250], [0.0208], [0.0]]
derink_categories = ['drink', 'drink', 'drink', 'drink']
drink_methodology = [
    '12 oz. cup of frozen concentrate, reconstituted with water makes 48 oz. of juice. A cup is 8 oz. of juice.',
    '8 oz. cup of Milk, fresh, whole, fortified resolved from gallon price',
    '96 teaspoons per 16 oz. of Coffee, 100%, ground roast, prepared with water, 2 teaspoon of coffee per cup',
    'Water is free'
]

In [36]:
side_ids = [10, 11, 12, 13, 14]
side_names = ['Cup of Yogurt', 'Cup of Fruit Cocktail', '2 Stips of Bacon', 'Hashbrowns', 'Toast']
side_item_codes = [['FJ4101'], ['711415', '711413', '711417'], ['704111'], ['712112'], ['702111']]
side_price_multipliers = [[0.5000], [0.1111, 0.0867, 0.0867], [0.1250], [0.1538], [0.0705]]
side_categories = ['side', 'side', 'side', 'side', 'side']
side_methodology = [
    'Yogurt, per 8 oz.',
    '1/2 cup of fruit cocktail from 1/6 cup of strawberries, 0.0867 lb for 1/6 cup of pears, 0.0867 lb for 1/6 cup of seedless grapes',
    '1/8 lb. of bacon as a strip from 1 lb. of bacon',
    '3 1/4 cups per pounds of potatoes, 1/2 cup of hashbrowns per serving',
    '32 grams of bread per slice, 1 slice of white bread per serving'
]

In [37]:
entree_ids = [20, 21, 22, 23]
entree_names = ['2-Pancakes', '2-Eggs', '16 oz. Pork Chop', 'Ham Egg and Cheese Melt']
entree_item_codes = [['701111', '709112', '708111'], ['708111'], ['FD3101'], ['702111', '704312', '710212']]
entree_categories = ['entree', 'entree', 'entree', 'entree']
entree_price_multipliers = [[0.0780, 0.0156, 0.0417], [0.1667], [1], [0.141, 0.1250, 0.1250]]
entree_methodology = [
    'pancake mix makes 4 servings, 1/4 cup of flour per serving, 1/4 cup of milk per serving, 1/4 grade A egg per serving',
    '2 grade A eggs per serving',
    '16 oz. Pork Chop per serving',
    '2 slices of white bread per serving, 1/8 lb. of ham per serving, 1/8 lb. of cheddar cheese per serving'
]

In [38]:
entrees = pd.DataFrame({'id': entree_ids, 'menu_name': entree_names, 'item_codes': entree_item_codes, 'price_multipliers': entree_price_multipliers, 'category': entree_categories, 'methodology': entree_methodology})
sides = pd.DataFrame({'id': side_ids, 'menu_name': side_names, 'item_codes': side_item_codes, 'price_multipliers': side_price_multipliers, 'category': side_categories, 'methodology': side_methodology})
drinks = pd.DataFrame({'id': drink_ids, 'menu_name': drink_names, 'item_codes': drink_item_codes, 'price_multipliers': drink_price_multipliers, 'category': derink_categories, 'methodology': drink_methodology})
menu = pd.concat([entrees, sides, drinks]).reset_index(drop=True)

In [39]:
def item_lookup(s):
    return items.query('item_name.str.contains("{}")'.format(s))

In [40]:
# a function to get the series ids from a list item codes
def get_series_ids(item_codes):
    series_ids = []
    for item_code in item_codes:
        series_ids.append(series.query('item_code == "{}"'.format(item_code)).series_id.tolist()[0])
    return series_ids

In [41]:
menu['series_id'] = menu.item_codes.apply(get_series_ids).reset_index(drop=True)

In [42]:
# converts a year and period to a datetime object
def yp_to_datetime(fulldata):
    years = []
    periods = []
    for i, row in fulldata.iterrows():
        year = int(row.year)
        period = int(str(row.period[1:]))
        years.append(year)
        periods.append(period)
    fulldata['datetime'] = pd.to_datetime({'year': years, 'month': periods, 'day': [1]*len(periods)})
    return fulldata

In [43]:
def unpack_items_and_timestep(menu, fulldata):
    menu_expanded = pd.DataFrame(columns=['id', 'menu_name', 'item_codes', 'price_multipliers', 'category', 'methodology', 'series_id', 'year', 'price'])
    ids = []
    menu_names = []
    item_codes = []
    price_multipliers = []
    categories = []
    methodologies = []
    series_ids = []
    years = []
    prices = []
    
    for i, row in menu.iterrows():
        for _, item in enumerate(row.item_codes):
            item_codes.append(item)
            ids.append(row.id)
            menu_names.append(row.menu_name)
            price_multipliers.append(row.price_multipliers[_])
            categories.append(row.category)
            methodologies.append(row.methodology)
            series_ids.append(row.series_id[_])
            
    unpacked_items = pd.DataFrame({'id': ids, 'menu_name': menu_names, 'item_codes': item_codes, 'price_multipliers': price_multipliers, 'category': categories, 'methodology': methodologies, 'series_id': series_ids})
    
    fulldata = yp_to_datetime(fulldata)
    
    fulldata = fulldata.join(unpacked_items.set_index('series_id'), on='series_id', how='inner')
    dt_frame = fulldata
    
    dt_frame = dt_frame[['series_id', 'value', 'datetime', 'id', 'price_multipliers']]
    
    dt_frame['price'] = dt_frame.value.apply(float) * dt_frame.price_multipliers
    
    dt_frame = dt_frame[['datetime', 'id', 'price']]
    
    return dt_frame.groupby(['id', 'datetime']).sum().reset_index()

In [44]:
menu_prices = unpack_items_and_timestep(menu, fulldata)

/tmp/ipykernel_171115/1270454476.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_frame['price'] = dt_frame.value.apply(float) * dt_frame.price_multipliers


In [45]:
menu[['id', 'menu_name', 'category', 'methodology']].to_csv('data/menu.csv', index=False)
menu[['id', 'menu_name', 'category', 'methodology']]


,id,menu_name,category,methodology
0,20,2-Pancakes,entree,"pancake mix makes 4 servings, 1/4 cup of flour..."
1,21,2-Eggs,entree,2 grade A eggs per serving
2,22,Pork Chop,entree,16 oz. Pork Chop per serving
3,23,Ham Egg and Cheese Melt,entree,"2 slices of white bread per serving, 1/8 lb. o..."
4,10,Cup of Yogurt,side,"Yogurt, per 8 oz."
5,11,Cup of Fruit Cocktail,side,1/2 cup of fruit cocktail from 1/6 cup of stra...
6,12,2 Stips of Bacon,side,1/8 lb. of bacon as a strip from 1 lb. of bacon
7,13,Hashbrowns,side,"3 1/4 cups per pounds of potatoes, 1/2 cup of ..."
8,14,Toast,side,"32 grams of bread per slice, 1 slice of white ..."
9,0,Cup of Orange Juice,drink,"12 oz. cup of frozen concentrate, reconstitute..."


In [46]:
menu_prices.to_csv('data/menu_prices.csv', index=False)
menu_prices

,id,datetime,price
0,0,1980-01-01,0.207208
1,0,1980-02-01,0.211876
2,0,1980-03-01,0.200207
3,0,1980-04-01,0.199873
4,0,1980-05-01,0.193705
...,...,...,...
5294,23,2022-11-01,1.692177
5295,23,2022-12-01,1.684468
5296,23,2023-01-01,1.694958
5297,23,2023-02-01,1.707086


In [47]:
fulldata['value'] = fulldata.value.apply(lambda x: str(x).strip()).apply(lambda x: x.replace('-', '0')).apply(float)

In [48]:
fulldata[['series_id', 'value', 'datetime']].to_csv('data/fulldata.csv', index=False)
fulldata[['series_id', 'value', 'datetime']]

,series_id,value,datetime
0,APU0000701111,0.203,1980-01-01
1,APU0000701111,0.205,1980-02-01
2,APU0000701111,0.211,1980-03-01
3,APU0000701111,0.206,1980-04-01
4,APU0000701111,0.207,1980-05-01
...,...,...,...
141780,APU0400FS1101,5.070,2022-11-01
141781,APU0400FS1101,5.296,2022-12-01
141782,APU0400FS1101,5.219,2023-01-01
141783,APU0400FS1101,5.129,2023-02-01


In [49]:
items.to_csv('data/items.csv', index=False)
items

,item_code,item_name
0,701111,"Flour, white, all purpose, per lb. (453.6 gm)"
1,701311,"Rice, white, long grain, precooked (cost per p..."
2,701312,"Rice, white, long grain, uncooked, per lb. (45..."
3,701321,Spaghetti (cost per pound/453.6 grams)
4,701322,"Spaghetti and macaroni, per lb. (453.6 gm)"
...,...,...
155,FJ4101,"Yogurt, per 8 oz. (226.8 gm)"
156,FL2101,"Lettuce, romaine, per lb. (453.6 gm)"
157,FN1101,"All soft drinks, per 2 liters (67.6 oz)"
158,FN1102,"All soft drinks, 12 pk, 12 oz., cans, per 12 o..."


In [50]:
def get_series_ids(item_codes):
    series_ids = []
    for item_code in [item_codes]:
        q = series.query('item_code == "{}"'.format(item_code)).series_id.tolist()
        series_ids.append(q[0])
    return series_ids[0]

In [51]:
items['series_id'] = items.item_code.apply(get_series_ids)

In [52]:
items.to_csv('data/items.csv', index=False)
items

,item_code,item_name,series_id
0,701111,"Flour, white, all purpose, per lb. (453.6 gm)",APU0000701111
1,701311,"Rice, white, long grain, precooked (cost per p...",APU0000701311
2,701312,"Rice, white, long grain, uncooked, per lb. (45...",APU0000701312
3,701321,Spaghetti (cost per pound/453.6 grams),APU0000701321
4,701322,"Spaghetti and macaroni, per lb. (453.6 gm)",APU0000701322
...,...,...,...
155,FJ4101,"Yogurt, per 8 oz. (226.8 gm)",APU0000FJ4101
156,FL2101,"Lettuce, romaine, per lb. (453.6 gm)",APU0000FL2101
157,FN1101,"All soft drinks, per 2 liters (67.6 oz)",APU0000FN1101
158,FN1102,"All soft drinks, 12 pk, 12 oz., cans, per 12 o...",APU0000FN1102


In [53]:
series

,series_id,area_code,item_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,APU0000701111,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2023,M03
1,APU0000701311,0000,701311,"Rice, white, long grain, precooked (cost per p...",NaN,1980,M01,1981,M12
2,APU0000701312,0000,701312,"Rice, white, long grain, uncooked, per lb. (45...",NaN,1980,M01,2023,M03
3,APU0000701321,0000,701321,Spaghetti (cost per pound/453.6 grams) in U.S....,NaN,1980,M01,1981,M03
4,APU0000701322,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",NaN,1984,M01,2023,M03
...,...,...,...,...,...,...,...,...,...
1477,APUS49G74713,S49G,74713,"Gasoline, leaded premium (cost per gallon/3.8 ...",NaN,1978,M01,1981,M04
1478,APUS49G74714,S49G,74714,"Gasoline, unleaded regular, per gallon/3.785 l...",NaN,1978,M01,2023,M03
1479,APUS49G74715,S49G,74715,"Gasoline, unleaded midgrade, per gallon/3.785 ...",NaN,2021,M06,2023,M03
1480,APUS49G74716,S49G,74716,"Gasoline, unleaded premium, per gallon/3.785 l...",NaN,1981,M09,2023,M03


In [14]:
fulldata = pd.read_csv('data/fulldata.csv')
fulldata['datetime'] = pd.to_datetime(fulldata.datetime)

In [5]:
import cpi

In [23]:
fulldata['adjusted_value'] = fulldata.apply(lambda x: cpi.inflate(x.value, x.datetime.year) if x.datetime.year<2023 else x.value, axis=1)

In [25]:
fulldata.to_csv('data/fulldata.csv', index=False)